# Missouri 2020 General Election Results

This notebook walks through wrangling, analysis and visualization of *unofficial* results of the statewide Missouri General Election, which will be held on November 3, 2020.

## Dependencies 

First, we import the Python modules (or parts of them) that our code will rely on. Here's what we need from Python's standard libary:

In [1]:
import os
import xml.etree.ElementTree as et

And here a couple of familiar third-party packages (previously installed via [pip](https://pip.pypa.io/en/stable/installing/)).

In [2]:
import pandas as pd
import altair as alt

And here is a third-party package that's new to our toolchain: [Requests](https://requests.readthedocs.io/en/master/), which is very useful for managing HTTP requests and responses.

In [3]:
import requests

[HTTP](https://developer.mozilla.org/en-US/docs/Web/HTTP) stands for **H**yper**t**ext **T**ransfer **P**rotocol, and it is the foundation of any data exchange on Web. It sets the rules for how clients like your preferred web browser (e.g., Firefox, Safari, Chrome) communicate with web servers that host web pages and other resources you might fetch.

The HTTP flow starts when a client sends a request to server. All requests have the following parts:

1. A [method](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods) indicating the user's desired action.
2. A path to a resource, indicated by Uniform Resource Locator (aka, [URL](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL)).
3. Optional headers that convey additional information to the server.

For more background, check out ["How the Web works"](https://developer.mozilla.org/en-US/docs/Learn/Getting_started_with_the_web/How_the_Web_works) and other resources on the Mozilla Developer Network.

## Accessing results data

For every state-wide election, the Missouri Secretary of State publishes county-level results. Initial results are posted after polls close at 7 pm on election night with updated results posted every few minutes until all precincts have reported.

Here is the URL:

In [4]:
results_url = "https://enrarchives.sos.mo.gov/apfeed/apfeed.asmx/GetElectionResults"

In order to access the feed, you need a key provided by SoS Elections Divisions. To keep the key hidden from public view, we can store it in a variable in our shell environment so that it won't show up in our notebook when we publish it.

Go to your launcher and launch a terminal session. Then you can set the environment variable:

```bash
set $ACCESS_KEY={paste the access key here}
```

Note that the environment variable will be deleted when the terminal session is closed. There are a lot of tools and tricks for storing project-specific environment variables and loading them as needed. My preference is [direnv](https://direnv.net/).

We can now access this environment variable via [`os`](https://docs.python.org/3/library/os.html), Python's built-in module for interacting with you computer's underlying operating system.

In [5]:
access_key = os.environ.get("ACCESS_KEY")

I can fetch this data using the popular [Requests](https://requests.readthedocs.io/en/master/) library, which provides a simple interface for sending [HTTP](https://developer.mozilla.org/en-US/docs/Web/HTTP) requests and processing the responses.

Here's how I make a `GET` request (the most common HTTP [request method](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods)) to the XML feed url. The access key is passed in via a query parameter.

In [6]:
r = requests.get(
    results_url,
    params={'AccessKey': access_key}
)

This method call returns a [`requests.Response`](https://requests.readthedocs.io/en/master/api/#requests.Response), which I've called `r`.

Here's how we check the general status of the response.

In [7]:
r.ok

False

Uh oh, it's not okay.

We can further investigate by checking the responses status code [status code](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status).

In [8]:
r.status_code

403

As is often the case, some web servers are configured to block requests coming from specific kinds of [user agents](https://developer.mozilla.org/en-US/docs/Glossary/user_agent) (i.e., software making a request on behalf of a user).

However, we can bypass this issue by modifying the [`'User-Agent'` header](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent) of the request. The default is `'python-requests/2.23.0'`, but we can substitute a header that a web browser would send. For example, here is user-agent header sent by the Firefox browser: `Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:83.0) Gecko/20100101 Firefox/83.0`.

Much like the parameters, the headers can be passed in as a `dict` through a keyword argument in the `.get` method call:

In [9]:
r = requests.get(
    results_url,
    params={'AccessKey': access_key},
    headers={'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:83.0) Gecko/20100101 Firefox/83.0"}
)

Now let's check the status again. This method will throw an exception if our response has a bad status.

In [10]:
r.ok

True

These election results are not permenantly available via this URL. Separate from this notebook, I've set up a process to archive these results every few minutes so that code in this notebook will always run.

In [11]:
try:
    r.raise_for_status()
except requests.HTTPError:
    r = requests.get("https://mo-gen-election-results-2020.s3.us-east-2.amazonaws.com/sos/latest.xml")

The actual data we're after is in the `.content` attibute of the response.

In [12]:
xml = r.content

## Preparing results data for analysis

Missouri's election results (and election results published by most other U.S. election authorities) are provided in [XML](https://developer.mozilla.org/en-US/docs/Web/XML/XML_introduction) format. XML stands for E**x**tensible **M**arkup **L**anguage, and it's closely related to another, more commonly known, Web technology: [HTML](https://developer.mozilla.org/en-US/docs/Learn/Getting_started_with_the_web/HTML_basics) (aka, **H**yper**t**ext **M**arkup **L**anguage).

HTML and XML both share the concept of [elements](https://developer.mozilla.org/en-US/docs/Glossary/element), which are the atomic units that define the structure of HTML and XML documents.

![](https://media.prod.mdn.mozit.cloud/attachments/2014/04/09/7659/a731e40efad1f6e0b728bfcf86c0035b/anatomy-of-an-html-element.png)

Whereas HTML has predefined tags like [`<h1>` through `<h6>`](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/Heading_Elements) for headings and [`<p>`](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/p) for paragraphs, XML allows data providers to define their own tags, which affords more semantic annotation of the document's content. In the sense, the tags in XML become sort of like column headers of a data table, communicate how we should interpret the inner content.

At the top of this notebook, we imported [`ElementTree`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#module-xml.etree.ElementTree)  under the alias `et` (this is part of Python's built-in [XML](https://docs.python.org/3.8/library/xml.etree.elementtree.html) module). In that module is a function named [`fromstring`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.fromstring), which parses a string of XML (i.e., the content of our response).

In [13]:
root = et.fromstring(xml)

The `fromstring` function returns the top-level or "root" [`Element`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element) of the XML. This is the outermost element of the XML tree.

Again, the name of each tag indicates the kind of data the element contains, sort of like a column header in a tabular data format. We can access this label like this:

In [14]:
root.tag

'ElectionResults'

We can also access the elements attributes, which is a dictionary of the all the stuff that's inside the opening tag. In this case, there's just one attribute: `'LastUpdated'`, which tells us the date and time the results were last updated.

In [15]:
root.attrib['LastUpdated']

'10/27/2020 04:51:42 PM'

We can also list the elements immediately inside a given element:

In [16]:
list(root)

[<Element 'ElectionInfo' at 0x120bf0810>]

The `Element` class has two other methods we'll make use of:

- [`.find`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element.find), which returns the *first* sub-element that matches the string you pass in.
- [`.findall`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element.findall), which returns *all* of the matching sub-elements in a list.

Both `.find` and `.findall` have one required argument: a string that matches the name of a tag on an immediate sub-element (aka, a "child" element):

In [17]:
root.find('ElectionInfo')

<Element 'ElectionInfo' at 0x120bf0810>

Or the path expressed in [XPath](https://docs.python.org/3.8/library/xml.etree.elementtree.html#elementtree-xpath). For example, this will return all of the elements with a tag named `<Counties>` and child element with a `<CountyName>` tag with "Boone" in the text.

In [18]:
root.findall(".//Counties[CountyName='Boone']")

[<Element 'Counties' at 0x120cbef90>,
 <Element 'Counties' at 0x120f37040>,
 <Element 'Counties' at 0x1211393b0>,
 <Element 'Counties' at 0x1212e21d0>,
 <Element 'Counties' at 0x12145b5e0>,
 <Element 'Counties' at 0x121592ae0>,
 <Element 'Counties' at 0x1216b5c70>,
 <Element 'Counties' at 0x1217c9270>,
 <Element 'Counties' at 0x12187e040>,
 <Element 'Counties' at 0x1218802c0>,
 <Element 'Counties' at 0x121880860>,
 <Element 'Counties' at 0x121880e00>,
 <Element 'Counties' at 0x121894d60>,
 <Element 'Counties' at 0x1219acc70>,
 <Element 'Counties' at 0x1219ce810>,
 <Element 'Counties' at 0x121aad630>,
 <Element 'Counties' at 0x121b269f0>,
 <Element 'Counties' at 0x121ba2c20>]

Unlike CSV and other tabular data formats, XML has a nested structure with elements inside other elements. We can't read XML directly into a `pandas.DataFrame` (though, I did just run across [this](https://pypi.org/project/pandas-read-xml/) third-party package, which looks quite promising).

Instead, we need to reformat the data from a nested structure into a two-dimensional structure that resembles a data table. We need to "flatten" the XML document's hierarchy such that each row in our data includes single observation with an observed value for each variable (i.e., columns). 

Essentially, we want each row to contain the number of votes counted for each ballot option in a given county and given contest.

Let's start by defining an empty list hold all of our rows

In [19]:
rows = []

Now let's work from the outside in.

In [20]:
for type_race in root.findall(".//TypeRace"):
    for race in type_race.findall("Race"):
        for counties in race.findall("Counties"):
            for party in counties.findall("CountyResults/Party"):
                for candidate in party.findall('Candidate'):
                    row = [
                        type_race.find('Type').text.strip(),
                        race.find('RaceTitle').text.strip(),
                        counties.find('CountyName').text.strip(),
                        int(counties.find('CountyResults/ReportingPrecincts').text),
                        int(counties.find('CountyResults/TotalPrecincts').text),
                        party.find('PartyName').text.strip(),
                        candidate.find('CandidateID').text.strip(),
                        candidate.find('LastName').text.strip(),
                        int(candidate.find('YesVotes').text),
                    ]                   
                    if candidate.find('NoVotes'):
                        row.append(
                            int(candidate.find('NoVotes').text)
                        )
                    else:
                        row.append(0)

                    rows.append(row)

In the innermost for loop, we define `row`, which contains the ordered column values for a given row of data. These values are extracted from the text of specific elements. For the few numeric values, we also coerce the strings to Python's `int` data type.

We also want to specify the column names and data_types (aka, [`dtypes`](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes)).

In [21]:
columns = {
    'type': "string",
    'race': "string",
    'county': "string",
    'reporting_precincts': "Int64",
    'total_precincts': "Int64",
    'party': "string",
    'candidate_id': "string",
    'candidate_name': "string",
    'yes_votes': "Int64",
    'no_votes': "Int64"
}

Now we can load these rows into a `DataFrame`. Note that we pass in the keys of the `columns` dict in order to specify the column names, and then the full mapping of column name and column data types when we call `.astype`.

In [22]:
results_df = pd.DataFrame(rows, columns=columns.keys()) \
    .astype(columns)

We also want to set the data types .

In [23]:
results_df

,type,race,county,reporting_precincts,total_precincts,party,candidate_id,candidate_name,yes_votes,no_votes
0,Federal,U.S. President and Vice President,Adair,0,12,Republican,750127734,"Donald J. Trump, Michael R. Pence",1,0
1,Federal,U.S. President and Vice President,Adair,0,12,Democratic,750127735,"Joseph R. Biden, Kamala D. Harris",2,0
2,Federal,U.S. President and Vice President,Adair,0,12,Libertarian,750127736,"Jo Jorgensen, Jeremy (Spike) Cohen",3,0
3,Federal,U.S. President and Vice President,Adair,0,12,Green,750127737,"Howie Hawkins, Angela Nicole Walker",4,0
4,Federal,U.S. President and Vice President,Adair,0,12,Constitution,750127738,"Don Blankenship, William Mohr",5,0
...,...,...,...,...,...,...,...,...,...,...
5786,Ballot Issues,Constitutional Amendment 3,Washington,0,14,Ballot Issues,750011439,Constitutional Amendment 3,6014,0
5787,Ballot Issues,Constitutional Amendment 3,Wayne,0,10,Ballot Issues,750011439,Constitutional Amendment 3,6016,0
5788,Ballot Issues,Constitutional Amendment 3,Webster,0,13,Ballot Issues,750011439,Constitutional Amendment 3,6018,0
5789,Ballot Issues,Constitutional Amendment 3,Worth,0,8,Ballot Issues,750011439,Constitutional Amendment 3,6020,0


I'll take a look at the columns, their positions, non-null counts and data-types:

In [24]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5791 entries, 0 to 5790
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   type                 5791 non-null   string
 1   race                 5791 non-null   string
 2   county               5791 non-null   string
 3   reporting_precincts  5791 non-null   Int64 
 4   total_precincts      5791 non-null   Int64 
 5   party                5791 non-null   string
 6   candidate_id         5791 non-null   string
 7   candidate_name       5791 non-null   string
 8   yes_votes            5791 non-null   Int64 
 9   no_votes             5791 non-null   Int64 
dtypes: Int64(4), string(6)
memory usage: 475.2 KB


And take a peek of the actual data:

In [25]:
results_df.head()

,type,race,county,reporting_precincts,total_precincts,party,candidate_id,candidate_name,yes_votes,no_votes
0,Federal,U.S. President and Vice President,Adair,0,12,Republican,750127734,"Donald J. Trump, Michael R. Pence",1,0
1,Federal,U.S. President and Vice President,Adair,0,12,Democratic,750127735,"Joseph R. Biden, Kamala D. Harris",2,0
2,Federal,U.S. President and Vice President,Adair,0,12,Libertarian,750127736,"Jo Jorgensen, Jeremy (Spike) Cohen",3,0
3,Federal,U.S. President and Vice President,Adair,0,12,Green,750127737,"Howie Hawkins, Angela Nicole Walker",4,0
4,Federal,U.S. President and Vice President,Adair,0,12,Constitution,750127738,"Don Blankenship, William Mohr",5,0


In [26]:
prez_results = results_df[results_df['race'] == "U.S. President and Vice President"]

In [27]:
prez_totals = prez_results \
    .groupby(['candidate_id', 'party', 'candidate_name']) \
    .yes_votes \
    .sum()

In [28]:
prez_totals.reset_index()

,candidate_id,party,candidate_name,yes_votes
0,750127734,Republican,"Donald J. Trump, Michael R. Pence",66816
1,750127735,Democratic,"Joseph R. Biden, Kamala D. Harris",66932
2,750127736,Libertarian,"Jo Jorgensen, Jeremy (Spike) Cohen",67048
3,750127737,Green,"Howie Hawkins, Angela Nicole Walker",67164
4,750127738,Constitution,"Don Blankenship, William Mohr",67280
5,750128582,Write-in,WRITE-INS,67396
6,750128584,Write-in,"Kasey Wells, Rachel Wells",67512
7,750128585,Write-in,"Brian Carroll, Amar Patel",67744
8,750128586,Write-in,"Gloria LaRiva, Sunil Freeman",67860
9,750128601,Write-in,"Jade Simmons, Claudeliah J. Roze",67628
